## Lesson 6. Start work with pg_probackup

### 1. Setup env variables. Get them from the last lesson

In [53]:
import os

os.environ['PG_CONFIG'] = '/usr/local/pgsql-17/bin/pg_config'
os.environ['PGPROBACKUPBIN'] = '/usr/local/pgsql-17/bin/pg_probackup'
os.environ['PGPROBACKUP_TMP_DIR'] = 'pg_probackup_demo'

### 2. Initialize pg_probackup2 environment

In [54]:
import testgres
from pg_probackup2.app import ProbackupApp
from pg_probackup2.init_helpers import Init
from pg_probackup2.storage.fs_backup import FSTestBackupDir


init_params = Init()
pg_node = testgres.NodeApp()

backup_dir = FSTestBackupDir(rel_path='rel_backup_dir', backup='backup_demo')
print(f"Backup catalog directory:\n{backup_dir.path}")

pb = ProbackupApp(
    pg_node=pg_node,
    pb_log_path=os.path.join(backup_dir.path, 'log'),
    backup_dir=backup_dir,
    probackup_path=init_params.probackup_path
)

Backup catalog directory:
/home/vshepard/PycharmProjects/backup_course/v6_pg_probackup_add_instance/pg_probackup_demo/rel_backup_dir/backup_demo


### 3. Create and start a PostgreSQL cluster

a. Create a simple PostgreSQL node using testgres
> initdb -D <data_directory>

In [55]:
node = pg_node.make_simple(
    base_dir=os.path.join(init_params.tmp_path, 'pg_node'),
    pg_options={
        "unix_socket_directories": "/tmp"
    }
)
node.status()

<NodeStatus.Stopped: 1>

b. Start the node
> pg_ctl start -D <data_directory>

In [56]:
node.slow_start()
print(f"PostgreSQL data directory: {node.data_dir}")
print(f"PostgreSQL port: {node.port}")
node.status()

PostgreSQL data directory: /home/vshepard/PycharmProjects/backup_course/v6_pg_probackup_add_instance/pg_probackup_demo/pg_node/data
PostgreSQL port: 55624


<NodeStatus.Running: 0>

### 4. Initialize the backup catalog

> pg_probackup init -B /mnt/backups

In [57]:
pb.init()

"INFO: Backup catalog '/home/vshepard/PycharmProjects/backup_course/v6_pg_probackup_add_instance/pg_probackup_demo/rel_backup_dir/backup_demo' successfully initialized\n"

### 5. Adding PostgreSQL instance to backup catalog

COMMON ERRORS

1. "ERROR: Instance already exists"
   - An instance with this name already exists
   - Use a different name or delete the existing instance

2. "ERROR: Cannot open directory"
   - The PGDATA path doesn't exist or is not accessible
   - Check the path and permissions

3. "ERROR: Backup catalog is not initialized"
   - Run pg_probackup init -B /mnt/backups first

> pg_probackup add-instance -B /mnt/backups -D /var/lib/pgpro/std-17/data --instance=main

In [58]:
pb.add_instance(instance='main', node=node)

"INFO: Instance 'main' successfully initialized\n"

### 6. Viewing instance configuration

> pg_probackup show-config -B /mnt/backups --instance=main

In [59]:
pb.show_config(instance='main')

{'pgdata': '/home/vshepard/PycharmProjects/backup_course/v6_pg_probackup_add_instance/pg_probackup_demo/pg_node/data',
 'system-identifier': '7604595754300462333',
 'xlog-seg-size': '16777216',
 'pgdatabase': 'postgres',
 'pgport': '55624',
 'replica-timeout': '5min',
 'archive-timeout': '5min',
 'log-level-console': 'INFO',
 'log-level-file': 'OFF',
 'log-format-console': 'PLAIN',
 'log-format-file': 'PLAIN',
 'log-filename': 'pg_probackup.log',
 'log-rotation-size': '0TB',
 'log-rotation-age': '0d',
 'retention-redundancy': '0',
 'retention-window': '0',
 'wal-depth': '0',
 'compress-algorithm': 'none',
 'compress-level': '1',
 'remote-proto': 'ssh'}

### 7. Configure instance settings

> pg_probackup set-config -B /mnt/backups --instance=main --pgport=<port>

In [60]:
pb.set_config(
    instance='main',
    options=[
        f'--pgport={node.port}',
        f'--pguser={init_params.username}',
        '--log-filename=my_log.txt',
        '--log-level-console=VERBOSE'
    ]
)

''

View updated configuaration
> pg_probackup show -B /mnt/backups – should show main

In [61]:
pb.show_config(instance='main')

{'pgdata': '/home/vshepard/PycharmProjects/backup_course/v6_pg_probackup_add_instance/pg_probackup_demo/pg_node/data',
 'system-identifier': '7604595754300462333',
 'xlog-seg-size': '16777216',
 'pgdatabase': 'postgres',
 'pgport': '55624',
 'pguser': 'vshepard',
 'replica-timeout': '5min',
 'archive-timeout': '5min',
 'log-level-console': 'VERBOSE',
 'log-level-file': 'OFF',
 'log-format-console': 'PLAIN',
 'log-format-file': 'PLAIN',
 'log-filename': 'my_log.txt',
 'log-rotation-size': '0TB',
 'log-rotation-age': '0d',
 'retention-redundancy': '0',
 'retention-window': '0',
 'wal-depth': '0',
 'compress-algorithm': 'none',
 'compress-level': '1',
 'remote-proto': 'ssh'}

### Create a backup

In [62]:
pb.backup_node(
    instance='main',
    node=node,
    backup_type='full',
    options=['--stream', f'--pguser={init_params.username}']
)

'TA5QP1'

Show backups
> pg_probackup show -B /mnt/backups

In [63]:
print(pb.show(as_text=True, as_json=False))


BACKUP INSTANCE 'main'
 Instance  Version  ID      Recovery Time           Mode  WAL Mode  TLI  Time  Data   WAL  Zratio  Start LSN  Stop LSN   Status 
 main      17       TA5QP1  2026-02-08 21:39:02+01  FULL  STREAM    1/0   10s  22MB  16MB    1,00  0/2000028  0/20001C0  OK     



### Cleanup

In [ ]:
node.stop()

import shutil
shutil.rmtree(os.environ['PGPROBACKUP_TMP_DIR'])

node.status()